In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import sys

sys.path.append(os.path.dirname(os.getcwd()))

import torch

from bertviz import head_view, model_view
from configs.base import Config
from transformers import BertTokenizer
from models.networks import MMSERALayerNorm
from data.dataloader import build_train_test_dataset

opt = Config()
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
device = torch.device("cpu")
# network =torch.load("/home/kuhaku/Code/EmotionClassification/code/3m-ser-private/scripts/checkpoints/MMSERALayerNorm/20230606-172654/weights/best_acc/checkpoint_0.pt",map_location=torch.device('cpu'),)
network = MMSERALayerNorm(num_classes=opt.num_classes, num_attention_head=opt.num_attention_head, dropout=opt.dropout)
train_ds, test_ds = build_train_test_dataset("/home/kuhaku/Code/EmotionClassification/code/3m-ser-private/scripts/data")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [29]:
for batch in test_ds:
    # Prepare batch
    text, label, sprectrome = batch["text"], batch["label"], batch["sprectrome"]
    label = torch.tensor([int(label)])
    input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0)

    # Move inputs to cpu or gpu
    sprectrome = sprectrome.to(device)
    label = label.to(device)
    input_ids = input_ids.to(device)
    with torch.no_grad():
        # Forward pass
        output, attention = network(input_ids, sprectrome, output_attentions=True)
        # Calculate accuracy
        _, preds = torch.max(output, 1)
        accuracy = torch.mean((preds == label).float())
    head_view([attention[0]], tokenizer.tokenize('[CLS] '+ text + ' [SEP]'))
    head_view([attention[1]], tokenizer.tokenize('[CLS] '+ text + ' [SEP]' + ''.join([' {}'.format(i) for i in range(sprectrome.shape[0])])))
    break

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
model_view([attention[0]], tokenizer.tokenize('[CLS] '+ text + ' [SEP]'))

<IPython.core.display.Javascript object>

In [ ]:
model_view([attention[1]], tokenizer.tokenize('[CLS] '+ text + ' [SEP]'))